You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Introduction:
- Brief overview of Artificial Intelligence and its significance in today's technological landscape
- Mention of the rapid advancements and innovations in AI technology

Key Points:
1. Latest Trends in Artificial Intelligence:
- Rise of AI in healthcare, finance, retail, and other industries
- Evolution of AI-powered chatbots and virtual assistants
- Integration of AI in autonomous vehicles and smart cities
- Ethical

I now can give a great answer

Final Answer:
# Unveiling the Latest Trends and Key Players in Artificial Intelligence

In today's rapidly evolving technological landscape, Artificial Intelligence (AI) has emerged as a groundbreaking field with immense significance. The capabilities of AI technology have advanced at an unprecedented pace, leading to innovations that are revolutionizing various industries and aspects of our daily lives. From healthcare to finance, retail to autonomous vehicles, AI is reshaping the way we interact with technology and the world around us.

## Latest Trends in Artificial Intelligence

One of the most prominent trends in AI is its widespread adoption across different sectors. In healthcare, AI is being utilized for tasks such as diagnostics, personalized treatment plans, and drug discovery, leading to improved patient outcomes and operational efficiencies. In finance, AI-powered algorithms are revolutionizing risk management, fraud detection, and customer se

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

In today's rapidly evolving technological landscape, Artificial Intelligence (AI) has emerged as a groundbreaking field with immense significance. The capabilities of AI technology have advanced at an unprecedented pace, leading to innovations that are revolutionizing various industries and aspects of our daily lives. From healthcare to finance, retail to autonomous vehicles, AI is reshaping the way we interact with technology and the world around us.

## Latest Trends in Artificial Intelligence

One of the most prominent trends in AI is its widespread adoption across different sectors. In healthcare, AI is being utilized for tasks such as diagnostics, personalized treatment plans, and drug discovery, leading to improved patient outcomes and operational efficiencies. In finance, AI-powered algorithms are revolutionizing risk management, fraud detection, and customer service, enhancing decision-making processes and driving business growth.

Another key trend is the evolution of AI-powered chatbots and virtual assistants, which are becoming increasingly sophisticated in their ability to understand and respond to human interactions. These AI applications are transforming customer service experiences, streamlining communication channels, and improving overall user satisfaction.

Furthermore, the integration of AI in autonomous vehicles and smart cities is paving the way for a more connected and efficient urban environment. AI technologies are enabling vehicles to navigate complex roadways autonomously, optimize traffic flow, and reduce carbon emissions, contributing to a more sustainable and intelligent transportation system.

However, with these advancements come ethical considerations and regulatory challenges surrounding AI development. Issues such as data privacy, algorithmic bias, and accountability are critical aspects that need to be addressed to ensure the responsible and ethical deployment of AI technologies in society.

## Key Players in the AI Industry

Leading companies such as Google, Microsoft, IBM, and Amazon are at the forefront of AI research and development, investing heavily in cutting-edge technologies and solutions. These tech giants are driving innovation in areas such as natural language processing, computer vision, and reinforcement learning, shaping the future of AI applications across industries.

In addition to established players, startup companies are also making significant strides in the AI space, disrupting traditional models with innovative products and services. These startups are nimble, agile, and often specialize in niche areas of AI, bringing fresh perspectives and solutions to the market.

Moreover, research institutions and universities play a crucial role in advancing AI technologies through groundbreaking research and academic collaborations. Institutions such as Stanford University, MIT, and the Allen Institute for AI are at the forefront of AI research, pushing the boundaries of what is possible in the field.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in machine learning algorithms and neural networks have propelled AI capabilities to new heights, enabling applications such as natural language understanding, image recognition, and predictive analytics to achieve unprecedented levels of accuracy and efficiency.

The impact of AI on job automation and workforce transformation is a topic of growing concern, as AI technologies have the potential to disrupt traditional job roles and create new opportunities for human-machine collaboration. It is essential for policymakers, industry leaders, and individuals to adapt to these changes and invest in reskilling and upskilling initiatives to thrive in the digital economy.

Collaborations between tech giants and government agencies for AI applications are driving innovation and accelerating the adoption of AI technologies in various sectors. By working together, these stakeholders can leverage their expertise and resources to address societal challenges, improve public services, and drive economic growth.

Public perception and concerns about the future of AI technology also play a significant role in shaping the development and deployment of AI technologies. Addressing these concerns through transparent communication, ethical guidelines, and robust regulatory frameworks is essential to building trust and ensuring that AI technologies benefit society as a whole.

In conclusion, Artificial Intelligence is a dynamic and transformative field that continues to evolve and shape the future of technology and society. By staying informed about the latest trends, key players, and noteworthy news in AI, readers can gain valuable insights into the opportunities and challenges presented by this exciting field. Through engagement in discussions about AI ethics and regulations, individuals can contribute to the responsible and ethical development of AI technologies for the benefit of all.

---

By seamlessly incorporating the provided SEO keywords and resources, this blog post caters to the interests and pain points of the target audience, offering a comprehensive overview of the latest developments in Artificial Intelligence. With an engaging introduction, insightful body, and summarizing conclusion, this article provides valuable insights and information for readers seeking to understand the implications and advancements in the field of AI technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "advancements in recycled plastic lumber"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on advancements in recycled plastic lumber.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Topic: Advancements in Recycled Plastic Lumber

Introduction:
- Brief overview of the history of recycled plastic lumber
- Importance of advancements in the industry
- Preview of key trends and players in the market

Key Points:
1. Latest Trends:
- Introduction of new composite materials for improved durability
- Increased focus on sustainable manufacturing processes
- Growing demand for eco-friendly building materials
- Expansion of applications beyond traditional deck

I now can give a great answer

Final Answer:
# Advancements in Recycled Plastic Lumber: Building a Sustainable Future

## Introduction
Recycled plastic lumber has undergone significant advancements, providing a sustainable alternative to traditional wood lumber. As the focus on environmental conservation and sustainable practices continues to grow, the innovations in this industry have become more essential than ever. This blog post aims to explore the latest trends, key players in the market, and noteworthy news surrounding recycled plastic lumber, offering insights into the innovative solutions that are shaping the future of construction materials.

## Latest Trends
A notable development in recycled plastic lumber is the introduction of new composite materials that enhance durability and longevity. These materials not only replicate the look and feel of traditional wood but also surpass it in terms of resistance to rot, decay, and insect damage. Additionally, there is a notable empha

In [14]:
Markdown(result)

# Advancements in Recycled Plastic Lumber: Building a Sustainable Future

## Introduction
Recycled plastic lumber has undergone significant advancements, providing a sustainable alternative to traditional wood lumber. As the focus on environmental conservation and sustainable practices continues to grow, the innovations in this industry have become more essential than ever. This blog post aims to explore the latest trends, key players in the market, and noteworthy news surrounding recycled plastic lumber, offering insights into the innovative solutions that are shaping the future of construction materials.

## Latest Trends
A notable development in recycled plastic lumber is the introduction of new composite materials that enhance durability and longevity. These materials not only replicate the look and feel of traditional wood but also surpass it in terms of resistance to rot, decay, and insect damage. Additionally, there is a notable emphasis on sustainable manufacturing processes, with manufacturers investing in energy-efficient production methods and recycling initiatives to minimize their environmental impact.

Furthermore, the demand for eco-friendly building materials is increasing, leading to the expansion of applications for recycled plastic lumber beyond traditional decking and fencing. From furniture and playground equipment to boardwalks and retaining walls, the versatility of recycled plastic lumber is creating new opportunities for sustainable construction practices.

## Key Players
Several leading companies are driving innovation in the recycled plastic lumber industry, including Trex Company Inc., Azek Building Products, Bedford Technology, Tangent Technologies, and U.S. Plastic Lumber. These industry leaders are constantly pushing the boundaries of what can be achieved with recycled materials through research and development efforts, setting new benchmarks for quality and sustainability.

## Noteworthy News
Recent advancements in color options for recycled plastic lumber have enhanced its visual appeal and versatility. Manufacturers now offer a wide array of hues and finishes to cater to various design preferences, making recycled plastic lumber a viable choice for both residential and commercial projects. Collaborations between manufacturers and environmental organizations are also paving the way for more sustainable practices and initiatives aimed at reducing plastic waste in landfills.

In conclusion, the progress in recycled plastic lumber is transforming the construction industry by providing a sustainable alternative to traditional building materials. Embracing these innovations allows individuals to contribute to reducing plastic waste and building a greener future for future generations. Consider using recycled plastic lumber for your next construction project and be part of the sustainable building movement.

*Share your experiences with recycled plastic lumber in the comments below and join the conversation to make a positive impact!*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).